In [1]:
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

In [2]:
import attr
import os
import pickle

from datetime import datetime, timedelta
from typing import List, Optional

@attr.s
class Activity:
    category: str = attr.ib()  # calendar
    start: datetime = attr.ib()
    end: datetime = attr.ib()
    name: Optional[str] = attr.ib(None)
    description: Optional[str] = attr.ib(None)

    @property
    def duration(self):
        "Return the duration of the activity in seconds."
        diff = self.end - self.start
        return diff.total_seconds()

In [3]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

def get_api_service(cache_file, credentials_file):
    creds = None

    if os.path.exists(cache_file):
        with open(cache_file, "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_file, SCOPES
            )
            creds = flow.run_local_server(port=0)

        with open(cache_file, "wb") as token:
            pickle.dump(creds, token)

    service = build("calendar", "v3", credentials=creds)
    return service

In [4]:
def get_calendar(name: str, cache_file, credentials_file):
    """Return the calendar ID."""
    service = get_api_service(cache_file, credentials_file)
    calendars_result = service.calendarList().list().execute()
    calendars = calendars_result.get("items", [])

    for calendar in calendars:
        if calendar.get("summary", "") == name:
            return calendar.get("id", None)

    return None

In [5]:
def load_calendar_data(
    calendars: List[str], 
    separator: str,
    credentials_file,
    cache_file,
    start_date: datetime = datetime.utcnow(), 
    end_date: datetime = datetime.utcnow()):
    return {
        calendar: load_data(
            calendar, 
            separator, 
            cache_file,
            credentials_file,
            start_date,
            end_date
        ) for calendar in calendars
    }

In [6]:
def load_data(calendar: str, separator: str, cache_file, credentials_file, start_date: datetime = datetime.utcnow(), end_date: datetime = datetime.utcnow()):
    name_description_separator = separator
    service = get_api_service(cache_file, credentials_file)

    calendar_id = get_calendar(calendar, cache_file, credentials_file)
    if calendar_id is None:
        print(f"Cannot find calendar {calendar}")
        return []

    start = start_date.isoformat() + "Z"
    end = end_date.isoformat() + "Z"
    events_request = service.events().list(
        calendarId=calendar_id,
        timeMin=start,
        timeMax=end,
        singleEvents=True,
    )

    processed_events = []

    while events_request is not None:
        events_result = (
            events_request.execute()
        )
        
        for event in events_result.get("items", []):
            category = calendar
            name = None
            description = None

            title = event["summary"].split(name_description_separator)

            if len(title) > 0:
                name = title[0]

            if len(title) > 1:
                description = title[1]
            try:
                start = datetime.strptime(
                    event["start"].get("dateTime", event["start"].get("date")),
                    '%Y-%m-%dT%H:%M:%S'
                )
                end = datetime.strptime(
                    event["end"].get("dateTime", event["end"].get("date")),
                    '%Y-%m-%dT%H:%M:%S'
                )

                if start.date() < end.date():
                    # split activity that goes over multiple days
                    processed_events.append(
                        Activity(
                            name=name,
                            description=description,
                            category=category,
                            start=start,
                            end=datetime.datetime.combine(start.date(), datetime.datetime.max.time()),
                        )
                    )
                    processed_events.append(
                        Activity(
                            name=name,
                            description=description,
                            category=category,
                            start=datetime.datetime.combine(end.date(), datetime.datetime.min.time()),
                            end=end,
                        )
                    )
                else:    
                    processed_events.append(
                        Activity(
                            name=name,
                            description=description,
                            category=category,
                            start=start,
                            end=end,
                        )
                    )
            except ValueError as v:
                ulr = len(v.args[0].partition('unconverted data remains: ')[2])
                if ulr:
                    start = datetime.strptime(
                        event["start"].get("dateTime", event["start"].get("date"))[:-ulr],
                        '%Y-%m-%dT%H:%M:%S'
                    )
                    end = datetime.strptime(
                        event["end"].get("dateTime", event["end"].get("date"))[:-ulr],
                        '%Y-%m-%dT%H:%M:%S'
                    )

                    if start.date() < end.date():
                        # split activity that goes over multiple days
                        processed_events.append(
                            Activity(
                                name=name,
                                description=description,
                                category=category,
                                start=start,
                                end=datetime.combine(start.date(), datetime.max.time()),
                            )
                        )
                        processed_events.append(
                            Activity(
                                name=name,
                                description=description,
                                category=category,
                                start=datetime.combine(end.date(), datetime.min.time()),
                                end=end,
                            )
                        )
                    else:    
                        processed_events.append(
                            Activity(
                                name=name,
                                description=description,
                                category=category,
                                start=start,
                                end=end,
                            )
                        )
                else:
                    pass

        events_request = service.events().list_next(events_request, events_result)

    return processed_events